# Exploratory Data Analysis (EDA) - [COUNTRY]

## Objective
Profile, clean, and explore the solar dataset to prepare it for comparison and region-ranking tasks.

## Instructions
1. Replace `[COUNTRY]` with your country name (e.g., Benin)
2. Update the data file path in the data loading section
3. Run all cells sequentially


## 1. Import Libraries and Load Data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Configuration
COUNTRY = "[COUNTRY]"  # Replace with your country name
DATA_PATH = "../data/[country]_raw.csv"  # Update with your data file path
OUTPUT_PATH = f"../data/{COUNTRY.lower()}_clean.csv"

print(f"Loading data for {COUNTRY}...")


In [ ]:
# Load the dataset
df = pd.read_csv(DATA_PATH)

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")
df.head()


In [ ]:
# Convert Timestamp to datetime if it's not already
if 'Timestamp' in df.columns:
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.sort_values('Timestamp').reset_index(drop=True)

df.info()


## 2. Summary Statistics & Missing Value Report


In [ ]:
# Summary statistics for all numeric columns
print("=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNumeric columns: {len(numeric_cols)}")
df[numeric_cols].describe()


In [ ]:
# Missing value analysis
print("=" * 80)
print("MISSING VALUE REPORT")
print("=" * 80)

missing_data = df.isna().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing Count': missing_data.values,
    'Missing Percentage': missing_percent.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Percentage', ascending=False)

print(f"\nTotal missing values: {missing_data.sum()}")
print(f"\nColumns with missing values:")
print(missing_df.to_string(index=False))

# Flag columns with >5% nulls
high_missing = missing_df[missing_df['Missing Percentage'] > 5]
if len(high_missing) > 0:
    print(f"\n⚠️  WARNING: Columns with >5% missing values:")
    print(high_missing[['Column', 'Missing Percentage']].to_string(index=False))
else:
    print("\n✅ No columns with >5% missing values")


In [ ]:
# Visualize missing values
if len(missing_df) > 0:
    plt.figure(figsize=(12, 6))
    missing_df_sorted = missing_df.sort_values('Missing Percentage', ascending=True)
    plt.barh(missing_df_sorted['Column'], missing_df_sorted['Missing Percentage'])
    plt.axvline(x=5, color='r', linestyle='--', label='5% threshold')
    plt.xlabel('Missing Percentage (%)')
    plt.ylabel('Columns')
    plt.title(f'Missing Values by Column - {COUNTRY}')
    plt.legend()
    plt.tight_layout()
    plt.show()


## 3. Outlier Detection & Basic Cleaning


In [ ]:
# Key columns for outlier detection
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Check which columns exist in the dataset
available_key_cols = [col for col in key_columns if col in df.columns]
print(f"Columns available for outlier detection: {available_key_cols}")

# Compute Z-scores for key columns
z_scores = pd.DataFrame()
outlier_flags = pd.DataFrame()

for col in available_key_cols:
    if col in numeric_cols:
        z_scores[col] = np.abs(stats.zscore(df[col].dropna()))
        outlier_flags[col] = z_scores[col] > 3

print(f"\nOutlier detection using Z-score (|Z| > 3):")
print("-" * 80)
for col in available_key_cols:
    if col in outlier_flags.columns:
        outlier_count = outlier_flags[col].sum()
        outlier_pct = (outlier_count / len(df)) * 100
        print(f"{col}: {outlier_count} outliers ({outlier_pct:.2f}%)")


In [ ]:
# Visualize outliers using box plots
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, col in enumerate(available_key_cols[:8]):  # Plot up to 8 columns
    if col in df.columns:
        df.boxplot(column=col, ax=axes[idx])
        axes[idx].set_title(f'{col} - Outliers')
        axes[idx].grid(True, alpha=0.3)

# Hide unused subplots
for idx in range(len(available_key_cols), 8):
    axes[idx].set_visible(False)

plt.suptitle(f'Outlier Detection - Box Plots for Key Variables - {COUNTRY}', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Handle missing values in key columns (impute with median)
key_columns_for_imputation = ['GHI', 'DNI', 'DHI', 'Tamb', 'RH', 'WS', 'ModA', 'ModB']
available_impute_cols = [col for col in key_columns_for_imputation if col in df_clean.columns]

print("Handling missing values:")
print("-" * 80)

for col in available_impute_cols:
    if col in numeric_cols:
        missing_before = df_clean[col].isna().sum()
        if missing_before > 0:
            median_val = df_clean[col].median()
            df_clean[col].fillna(median_val, inplace=True)
            print(f"{col}: Imputed {missing_before} missing values with median ({median_val:.2f})")

print(f"\nMissing values after imputation: {df_clean.isna().sum().sum()}")

# Cap outliers at 3 standard deviations for key columns
critical_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
for col in critical_cols:
    if col in df_clean.columns and col in numeric_cols:
        mean_val = df_clean[col].mean()
        std_val = df_clean[col].std()
        df_clean[col] = df_clean[col].clip(lower=mean_val - 3*std_val, upper=mean_val + 3*std_val)

print(f"\nOutliers capped at ±3 standard deviations")


## 4. Time Series Analysis


In [ ]:
# Helper function to get units
def get_units(var):
    """Helper function to get units for variables"""
    units = {
        'GHI': 'W/m²', 'DNI': 'W/m²', 'DHI': 'W/m²',
        'ModA': 'W/m²', 'ModB': 'W/m²',
        'Tamb': '°C', 'TModA': '°C', 'TModB': '°C',
        'RH': '%', 'WS': 'm/s', 'WSgust': 'm/s',
        'BP': 'hPa', 'Precipitation': 'mm/min'
    }
    return units.get(var, '')

# Ensure Timestamp is the index for time series analysis
if 'Timestamp' in df_clean.columns:
    df_clean['Date'] = df_clean['Timestamp'].dt.date
    df_clean['Month'] = df_clean['Timestamp'].dt.month
    df_clean['Hour'] = df_clean['Timestamp'].dt.hour
    df_clean['DayOfYear'] = df_clean['Timestamp'].dt.dayofyear

# Time series plots for key variables
ts_vars = ['GHI', 'DNI', 'DHI', 'Tamb']
available_ts_vars = [col for col in ts_vars if col in df_clean.columns]

fig, axes = plt.subplots(len(available_ts_vars), 1, figsize=(16, 4*len(available_ts_vars)))
if len(available_ts_vars) == 1:
    axes = [axes]

for idx, var in enumerate(available_ts_vars):
    if 'Timestamp' in df_clean.columns:
        axes[idx].plot(df_clean['Timestamp'], df_clean[var], alpha=0.6, linewidth=0.5)
        axes[idx].set_title(f'{var} Over Time - {COUNTRY}', fontsize=14)
        axes[idx].set_xlabel('Timestamp')
        axes[idx].set_ylabel(f'{var} ({get_units(var)})')
        axes[idx].grid(True, alpha=0.3)
        plt.setp(axes[idx].xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()


In [ ]:
# Monthly patterns
if 'Month' in df_clean.columns and 'GHI' in df_clean.columns:
    monthly_avg = df_clean.groupby('Month')[available_ts_vars].mean()
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    for idx, var in enumerate(available_ts_vars[:4]):
        axes[idx].bar(range(1, 13), monthly_avg[var], color='steelblue', alpha=0.7)
        axes[idx].set_title(f'Average {var} by Month - {COUNTRY}', fontsize=12)
        axes[idx].set_xlabel('Month')
        axes[idx].set_ylabel(f'{var} ({get_units(var)})')
        axes[idx].set_xticks(range(1, 13))
        axes[idx].set_xticklabels(month_names, rotation=45)
        axes[idx].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Hourly patterns (diurnal cycle)
if 'Hour' in df_clean.columns and 'GHI' in df_clean.columns:
    hourly_avg = df_clean.groupby('Hour')[available_ts_vars].mean()
    
    fig, ax = plt.subplots(figsize=(14, 6))
    for var in available_ts_vars:
        ax.plot(hourly_avg.index, hourly_avg[var], marker='o', label=var, linewidth=2)
    
    ax.set_title(f'Average Diurnal Cycle (Hourly Patterns) - {COUNTRY}', fontsize=14)
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Average Value')
    ax.set_xticks(range(0, 24, 2))
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


## 5. Cleaning Impact Analysis


In [ ]:
# Analyze impact of cleaning events on ModA and ModB
if 'Cleaning' in df_clean.columns:
    cleaning_analysis = df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()
    
    print("Cleaning Impact Analysis:")
    print("=" * 80)
    print(cleaning_analysis)
    
    # Visualize cleaning impact
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for idx, mod in enumerate(['ModA', 'ModB']):
        if mod in df_clean.columns:
            cleaning_analysis[mod].plot(kind='bar', ax=axes[idx], color=['lightcoral', 'lightgreen'])
            axes[idx].set_title(f'Average {mod} by Cleaning Status - {COUNTRY}')
            axes[idx].set_xlabel('Cleaning (0=No, 1=Yes)')
            axes[idx].set_ylabel(f'{mod} (W/m²)')
            axes[idx].set_xticklabels(['No Cleaning', 'Cleaning'], rotation=0)
            axes[idx].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Before/After cleaning comparison
    if len(df_clean[df_clean['Cleaning'] == 1]) > 0:
        print("\nDetailed Cleaning Analysis:")
        print("-" * 80)
        for mod in ['ModA', 'ModB']:
            if mod in df_clean.columns:
                before = df_clean[df_clean['Cleaning'] == 0][mod].mean()
                after = df_clean[df_clean['Cleaning'] == 1][mod].mean()
                improvement = ((after - before) / before) * 100
                print(f"{mod}: Before={before:.2f}, After={after:.2f}, Improvement={improvement:.2f}%")


## 6. Correlation & Relationship Analysis


In [ ]:
# Correlation analysis
corr_vars = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB', 'Tamb', 'RH', 'WS', 'WSgust', 'BP']
available_corr_vars = [col for col in corr_vars if col in df_clean.columns and col in numeric_cols]

if len(available_corr_vars) > 0:
    corr_matrix = df_clean[available_corr_vars].corr()
    
    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title(f'Correlation Heatmap - {COUNTRY}', fontsize=16, pad=20)
    plt.tight_layout()
    plt.show()
    
    # Print strongest correlations
    print("\nStrongest Correlations (|r| > 0.7):")
    print("-" * 80)
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_val = corr_matrix.iloc[i, j]
            if abs(corr_val) > 0.7:
                print(f"{corr_matrix.columns[i]} vs {corr_matrix.columns[j]}: {corr_val:.3f}")


In [ ]:
# Scatter plots: WS, WSgust, WD vs. GHI
if 'GHI' in df_clean.columns:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    scatter_vars = [('WS', 'Wind Speed'), ('WSgust', 'Wind Gust Speed'), ('WD', 'Wind Direction')]
    
    for idx, (var, label) in enumerate(scatter_vars):
        if var in df_clean.columns:
            axes[idx].scatter(df_clean[var], df_clean['GHI'], alpha=0.3, s=10)
            axes[idx].set_xlabel(f'{label} ({get_units(var)})')
            axes[idx].set_ylabel(f'GHI (W/m²)')
            axes[idx].set_title(f'GHI vs {label} - {COUNTRY}')
            axes[idx].grid(True, alpha=0.3)
            
            # Add correlation coefficient
            corr_coef = df_clean[[var, 'GHI']].corr().iloc[0, 1]
            axes[idx].text(0.05, 0.95, f'r = {corr_coef:.3f}', 
                          transform=axes[idx].transAxes, 
                          verticalalignment='top',
                          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Scatter plots: RH vs. Tamb and RH vs. GHI
if 'RH' in df_clean.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # RH vs Tamb
    if 'Tamb' in df_clean.columns:
        axes[0].scatter(df_clean['RH'], df_clean['Tamb'], alpha=0.3, s=10, color='blue')
        axes[0].set_xlabel('Relative Humidity (%)')
        axes[0].set_ylabel('Ambient Temperature (°C)')
        axes[0].set_title(f'RH vs Tamb - {COUNTRY}')
        axes[0].grid(True, alpha=0.3)
        corr_rh_tamb = df_clean[['RH', 'Tamb']].corr().iloc[0, 1]
        axes[0].text(0.05, 0.95, f'r = {corr_rh_tamb:.3f}', 
                    transform=axes[0].transAxes, 
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # RH vs GHI
    if 'GHI' in df_clean.columns:
        axes[1].scatter(df_clean['RH'], df_clean['GHI'], alpha=0.3, s=10, color='orange')
        axes[1].set_xlabel('Relative Humidity (%)')
        axes[1].set_ylabel('GHI (W/m²)')
        axes[1].set_title(f'RH vs GHI - {COUNTRY}')
        axes[1].grid(True, alpha=0.3)
        corr_rh_ghi = df_clean[['RH', 'GHI']].corr().iloc[0, 1]
        axes[1].text(0.05, 0.95, f'r = {corr_rh_ghi:.3f}', 
                    transform=axes[1].transAxes, 
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()


## 7. Wind & Distribution Analysis


In [ ]:
# Wind Rose (Radial Bar Plot)
if 'WD' in df_clean.columns and 'WS' in df_clean.columns:
    # Create wind direction bins (16 directions)
    df_clean['WD_bin'] = pd.cut(df_clean['WD'], bins=16, labels=range(16))
    
    # Calculate average wind speed by direction
    wind_rose = df_clean.groupby('WD_bin')['WS'].mean()
    
    # Create radial plot
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    # Convert bin numbers to angles (in radians)
    angles = np.linspace(0, 2*np.pi, 16, endpoint=False)
    
    # Plot bars
    bars = ax.bar(angles, wind_rose.values, width=2*np.pi/16, color='steelblue', alpha=0.7)
    
    # Set direction labels
    direction_labels = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
                       'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    ax.set_xticks(angles)
    ax.set_xticklabels(direction_labels)
    
    ax.set_title(f'Wind Rose - Average Wind Speed by Direction - {COUNTRY}', 
                fontsize=14, pad=20)
    ax.set_ylabel('Wind Speed (m/s)', labelpad=30)
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Histograms for GHI and WS
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# GHI Histogram
if 'GHI' in df_clean.columns:
    axes[0].hist(df_clean['GHI'].dropna(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('GHI (W/m²)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(f'Distribution of GHI - {COUNTRY}')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Add statistics
    mean_ghi = df_clean['GHI'].mean()
    median_ghi = df_clean['GHI'].median()
    axes[0].axvline(mean_ghi, color='red', linestyle='--', label=f'Mean: {mean_ghi:.1f}')
    axes[0].axvline(median_ghi, color='green', linestyle='--', label=f'Median: {median_ghi:.1f}')
    axes[0].legend()

# WS Histogram
if 'WS' in df_clean.columns:
    axes[1].hist(df_clean['WS'].dropna(), bins=50, color='orange', alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Wind Speed (m/s)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'Distribution of Wind Speed - {COUNTRY}')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Add statistics
    mean_ws = df_clean['WS'].mean()
    median_ws = df_clean['WS'].median()
    axes[1].axvline(mean_ws, color='red', linestyle='--', label=f'Mean: {mean_ws:.2f}')
    axes[1].axvline(median_ws, color='green', linestyle='--', label=f'Median: {median_ws:.2f}')
    axes[1].legend()

plt.tight_layout()
plt.show()


## 8. Temperature Analysis


In [ ]:
# Temperature and RH relationship analysis
if 'Tamb' in df_clean.columns and 'RH' in df_clean.columns:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. RH vs Tamb scatter with density
    axes[0, 0].hexbin(df_clean['RH'], df_clean['Tamb'], gridsize=30, cmap='YlOrRd')
    axes[0, 0].set_xlabel('Relative Humidity (%)')
    axes[0, 0].set_ylabel('Ambient Temperature (°C)')
    axes[0, 0].set_title(f'RH vs Tamb (Density) - {COUNTRY}')
    plt.colorbar(axes[0, 0].collections[0], ax=axes[0, 0])
    
    # 2. RH vs GHI
    if 'GHI' in df_clean.columns:
        axes[0, 1].hexbin(df_clean['RH'], df_clean['GHI'], gridsize=30, cmap='Blues')
        axes[0, 1].set_xlabel('Relative Humidity (%)')
        axes[0, 1].set_ylabel('GHI (W/m²)')
        axes[0, 1].set_title(f'RH vs GHI (Density) - {COUNTRY}')
        plt.colorbar(axes[0, 1].collections[0], ax=axes[0, 1])
    
    # 3. Temperature distribution by RH levels
    df_clean['RH_category'] = pd.cut(df_clean['RH'], bins=[0, 30, 60, 100], 
                                     labels=['Low (0-30%)', 'Medium (30-60%)', 'High (60-100%)'])
    if 'RH_category' in df_clean.columns:
        df_clean.boxplot(column='Tamb', by='RH_category', ax=axes[1, 0])
        axes[1, 0].set_title(f'Tamb Distribution by RH Category - {COUNTRY}')
        axes[1, 0].set_xlabel('RH Category')
        axes[1, 0].set_ylabel('Ambient Temperature (°C)')
    
    # 4. GHI distribution by RH levels
    if 'GHI' in df_clean.columns and 'RH_category' in df_clean.columns:
        df_clean.boxplot(column='GHI', by='RH_category', ax=axes[1, 1])
        axes[1, 1].set_title(f'GHI Distribution by RH Category - {COUNTRY}')
        axes[1, 1].set_xlabel('RH Category')
        axes[1, 1].set_ylabel('GHI (W/m²)')
    
    plt.tight_layout()
    plt.show()
    
    # Statistical summary
    print("Temperature and RH Relationship:")
    print("=" * 80)
    if 'RH_category' in df_clean.columns:
        summary = df_clean.groupby('RH_category')[['Tamb', 'GHI']].agg(['mean', 'std'])
        print(summary)


## 9. Bubble Chart


In [ ]:
# Bubble Chart: GHI vs. Tamb with bubble size = RH or BP
if 'GHI' in df_clean.columns and 'Tamb' in df_clean.columns:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Sample data for better visualization (if dataset is too large)
    sample_size = min(5000, len(df_clean))
    df_sample = df_clean.sample(n=sample_size, random_state=42)
    
    # Bubble chart with RH as bubble size
    if 'RH' in df_clean.columns:
        scatter1 = axes[0].scatter(df_sample['Tamb'], df_sample['GHI'], 
                                  s=df_sample['RH']*5, alpha=0.5, 
                                  c=df_sample['RH'], cmap='viridis')
        axes[0].set_xlabel('Ambient Temperature (°C)')
        axes[0].set_ylabel('GHI (W/m²)')
        axes[0].set_title(f'GHI vs Tamb (Bubble Size = RH) - {COUNTRY}')
        axes[0].grid(True, alpha=0.3)
        plt.colorbar(scatter1, ax=axes[0], label='RH (%)')
    
    # Bubble chart with BP as bubble size
    if 'BP' in df_clean.columns:
        # Normalize BP for bubble size (scale to reasonable range)
        bp_normalized = (df_sample['BP'] - df_sample['BP'].min()) / (df_sample['BP'].max() - df_sample['BP'].min()) * 200 + 10
        scatter2 = axes[1].scatter(df_sample['Tamb'], df_sample['GHI'], 
                                  s=bp_normalized, alpha=0.5, 
                                  c=df_sample['BP'], cmap='plasma')
        axes[1].set_xlabel('Ambient Temperature (°C)')
        axes[1].set_ylabel('GHI (W/m²)')
        axes[1].set_title(f'GHI vs Tamb (Bubble Size = BP) - {COUNTRY}')
        axes[1].grid(True, alpha=0.3)
        plt.colorbar(scatter2, ax=axes[1], label='BP (hPa)')
    
    plt.tight_layout()
    plt.show()


## 10. Summary & Export Cleaned Data


In [ ]:
# Final summary
print("=" * 80)
print(f"EDA SUMMARY FOR {COUNTRY}")
print("=" * 80)
print(f"\nOriginal dataset shape: {df.shape}")
print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Rows removed: {df.shape[0] - df_clean.shape[0]}")
print(f"Columns: {df_clean.shape[1]}")

print(f"\nMissing values:")
print(f"  Before cleaning: {df.isna().sum().sum()}")
print(f"  After cleaning: {df_clean.isna().sum().sum()}")

if 'GHI' in df_clean.columns:
    print(f"\nKey Statistics (GHI):")
    print(f"  Mean: {df_clean['GHI'].mean():.2f} W/m²")
    print(f"  Median: {df_clean['GHI'].median():.2f} W/m²")
    print(f"  Std: {df_clean['GHI'].std():.2f} W/m²")
    print(f"  Min: {df_clean['GHI'].min():.2f} W/m²")
    print(f"  Max: {df_clean['GHI'].max():.2f} W/m²")


In [ ]:
# Remove temporary columns before exporting
cols_to_drop = ['Date', 'Month', 'Hour', 'DayOfYear', 'WD_bin', 'RH_category']
export_cols = [col for col in cols_to_drop if col in df_clean.columns]
df_export = df_clean.drop(columns=export_cols, errors='ignore')

# Export cleaned data
import os
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
df_export.to_csv(OUTPUT_PATH, index=False)
print(f"\n✅ Cleaned data exported to: {OUTPUT_PATH}")
print(f"   File size: {os.path.getsize(OUTPUT_PATH) / 1024 / 1024:.2f} MB")


## References & Insights

### Key Insights:
1. **Data Quality**: [Add your observations about data quality]
2. **Solar Patterns**: [Add insights about solar radiation patterns]
3. **Weather Impact**: [Add insights about weather variables]
4. **Cleaning Impact**: [Add insights about cleaning events]

### Statistical Distributions Observed:
- [Add observations about distributions]

### Actionable Insights:
- [Add actionable insights gained from EDA]
